In [9]:
!pip install primefac

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
     |████████████████████████████████| 133 kB 3.4 MB/s            
/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [13]:
import uproot
import awkward as ak
import json

In [11]:
# Generate prime numbers and divisors up to size n.
from docs import gen_primes
n = 100
gen_primes.writer(n, dir='docs')

In [43]:
# Let's Look at our file's structure!

with open('utilities/prime_factors.json') as file:
    data = json.load(file)

In [44]:
data

{'0': {'prime_factors': [], 'unique_divisors': []},
 '1': {'prime_factors': [], 'unique_divisors': [1]},
 '2': {'prime_factors': [2], 'unique_divisors': [1, 2]},
 '3': {'prime_factors': [3], 'unique_divisors': [1, 3]},
 '4': {'prime_factors': [2, 2], 'unique_divisors': [1, 2, 4]},
 '5': {'prime_factors': [5], 'unique_divisors': [1, 5]},
 '6': {'prime_factors': [2, 3], 'unique_divisors': [1, 2, 3, 6]},
 '7': {'prime_factors': [7], 'unique_divisors': [1, 7]},
 '8': {'prime_factors': [2, 2, 2], 'unique_divisors': [1, 2, 4, 8]},
 '9': {'prime_factors': [3, 3], 'unique_divisors': [1, 3, 9]},
 '10': {'prime_factors': [2, 5], 'unique_divisors': [1, 2, 5, 10]},
 '11': {'prime_factors': [11], 'unique_divisors': [1, 11]},
 '12': {'prime_factors': [2, 2, 3], 'unique_divisors': [1, 2, 3, 4, 6, 12]},
 '13': {'prime_factors': [13], 'unique_divisors': [1, 13]},
 '14': {'prime_factors': [2, 7], 'unique_divisors': [1, 2, 7, 14]},
 '15': {'prime_factors': [3, 5], 'unique_divisors': [1, 3, 5, 15]},
 '16'

In [57]:
# Okay. Let's say we want to select only numbers that have 3 as a prime factor and at least four unique divisors. The event loop method would be:
cut = []
for number in data:
    if (3 in data[number]['prime_factors']) & (len(data[number]['unique_divisors']) > 4):
        cut.append((number, data[number]['unique_divisors']))
        
cut

[('12', [1, 2, 3, 4, 6, 12]),
 ('18', [1, 2, 3, 6, 9, 18]),
 ('24', [1, 2, 3, 4, 6, 8, 12, 24]),
 ('30', [1, 2, 3, 5, 6, 10, 15, 30]),
 ('36', [1, 2, 3, 4, 6, 9, 12, 18, 36]),
 ('42', [1, 2, 3, 6, 7, 14, 21, 42]),
 ('45', [1, 3, 5, 9, 15, 45]),
 ('48', [1, 2, 3, 4, 6, 8, 12, 16, 24, 48]),
 ('54', [1, 2, 3, 6, 9, 18, 27, 54]),
 ('60', [1, 2, 3, 4, 5, 6, 10, 12, 15, 20, 30, 60]),
 ('63', [1, 3, 7, 9, 21, 63]),
 ('66', [1, 2, 3, 6, 11, 22, 33, 66]),
 ('72', [1, 2, 3, 4, 6, 8, 9, 12, 18, 24, 36, 72]),
 ('75', [1, 3, 5, 15, 25, 75]),
 ('78', [1, 2, 3, 6, 13, 26, 39, 78]),
 ('81', [1, 3, 9, 27, 81]),
 ('84', [1, 2, 3, 4, 6, 7, 12, 14, 21, 28, 42, 84]),
 ('90', [1, 2, 3, 5, 6, 9, 10, 15, 18, 30, 45, 90]),
 ('96', [1, 2, 3, 4, 6, 8, 12, 16, 24, 32, 48, 96]),
 ('99', [1, 3, 9, 11, 33, 99])]

In [66]:
# That's not too bad. But with more fields and higher dimensions, as we have in high-energy physics, it can get a little complicated. Let's look at an alternative: columnar analysis.
# Awkward is a package which behaves a lot like numpy, but accepts jagged data.

data = ak.from_json('docs/prime_factors.json', highlevel=False)

In [68]:
# What does this look like?
data

<Record at="0">
    <RecordArray length="1">
        <field index="0" key="0">
            <RecordArray length="1">
                <field index="0" key="prime_factors">
                    <ListOffsetArray64>
                        <offsets><Index64 i="[0 0]" offset="0" length="2" at="0x5624c47f9870"/></offsets>
                        <content><EmptyArray/></content>
                    </ListOffsetArray64>
                </field>
                <field index="1" key="unique_divisors">
                    <ListOffsetArray64>
                        <offsets><Index64 i="[0 0]" offset="0" length="2" at="0x5624c47f8bc0"/></offsets>
                        <content><EmptyArray/></content>
                    </ListOffsetArray64>
                </field>
            </RecordArray>
        </field>
        <field index="1" key="1">
            <RecordArray length="1">
                <field index="0" key="prime_factors">
                    <ListOffsetArray64>
                        <of

In [5]:
load_file = ak.from_json('docs/prime_factors.json')

In [6]:
load_file.ke

AttributeError: no field named 'ke'

(https://github.com/scikit-hep/awkward-1.0/blob/1.8.0/src/awkward/highlevel.py#L1846)

In [ ]:
# Let's say we want all numbers with more than 4 prime factors. The event loop solution to this would be:
four_primes = []
for i in load_file:
    if len(load_file['prime_factors'][i]) > 4:
        four_primes.append(i)
four_primes

In [ ]:
# The columnar solution would be:
load_file['prime_factors'][3 in load_file['prime_factors']]

In [ ]:
file = uproot.open("root://eospublic.cern.ch//eos/root-eos/benchmark/Run2012B_SingleMu.root")
events = file['Events']